In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.relative_locator import locate_with
from selenium.webdriver.common.by import By
import os
import re
from time import sleep

upper_bound = 15409

cwd = os.getcwd()

download_dir = os.path.join(cwd, "OR_Storage")
editAddress = os.path.join(download_dir, "OfficialReport.pdf")

options = Options()
options.page_load_strategy = 'normal'
options.add_argument("--incognito")
options.headless = True
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)

driver_path = os.path.join("C:\\Chrome Drivers\\V116\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(executable_path=driver_path, options=options)
base_url = "https://www.parliament.scot/api/sitecore/CustomMedia/OfficialReport?meetingId="

regex1 = "Error"
regex2 = "404"

In [10]:
driver.get("https://www.parliament.scot/api/sitecore/CustomMedia/OfficialReport?meetingId=300")


In [11]:
driver.current_url

'https://www.parliament.scot/api/sitecore/CustomMedia/OfficialReport?meetingId=5899'

In [12]:
def OffRepGatherer(base_url, batch_no):
    tracker = "fileprogress.txt"
    
    if os.path.isfile(tracker):
        with open(tracker, 'r') as file:
            start_no = int(file.read())
            print(start_no)
    else:
        start_no = 1
        
    end_no = start_no + batch_no
    for i in range(start_no, end_no):
        renamed = True
        newUrl = base_url + str(i)
        try:
            driver.get(newUrl)
            
        except:
            with open("OffRep_ErrorLog.txt", 'a') as el:
                error_log = "Unable to pull document from " + newUrl + "\n"
                el.write(error_log)
        
        if driver.current_url == newUrl:
            with open("OffRep_ErrorLog.txt", 'a') as el:
                error_log = "Document " + str(i) + " not available from API\n"
                el.write(error_log)
                
            if i > upper_bound:
                return True
        
        else:
            
            attempts = 0
            while renamed:
                if os.path.isfile(editAddress):
                    newName = os.path.join(download_dir+("\\OR-" + str(i)+".pdf"))
                    os.rename(editAddress, newName)
                    renamed = False
                else:
                    if attempts > 30:
                        renamed = False
                        with open("OffRep_ErrorLog.txt", 'a') as el:
                            error_log = "Error renaming document downloaded from:\n " + newUrl+"\n"
                            el.write(error_log)
                    else:
                        attempts += 1
                        sleep(3)
    

    with open(tracker, 'w') as file:
        end_no = str(end_no)
        file.write(end_no)
        
    return False
        
            
            
            

In [15]:
OffRepGatherer(base_url, 10000)

6243


True

In [14]:
tracker = "fileprogress.txt"
os.path.isfile(tracker)

True